In [1]:
import numpy as np
import csv
import keras
import sklearn
import gensim
import random
import scipy
import pandas as pd
import emoji
import re
from keras.preprocessing import text
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers.core import Dense , Dropout , Activation
from keras.layers import Embedding , LSTM, BatchNormalization, SpatialDropout1D, Bidirectional
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report 
from gensim.models.word2vec import Word2Vec
from gensim.models.doc2vec import Doc2Vec , TaggedDocument
from keras.utils.np_utils import to_categorical
from sklearn.utils import class_weight

# size of the word embeddings
embeddings_dim = 300

# maximum number of words to consider in the representations
max_features = 15000

# maximum length of a sentence
max_sent_len = 20

# percentage of the data used for model training
percent = 0.70

# number of classes
num_classes = 4

embeddings = dict( )

#training dataset
data = [ ( row["single"] , row["label"]  ) for row in csv.DictReader(open("C:\\Users\\Premnath\\Desktop\\MDSS\\MDDS 9\\train.txt",encoding='utf8'), delimiter='\t', quoting=csv.QUOTE_NONE) ]
random.shuffle( data )
train_size = int(len(data) * percent)
       
emo_train_texts = [ txt.lower() for ( txt, label ) in data[0:train_size] ]
train_labels = [ label for ( txt , label ) in data[0:train_size] ]

emo_test_texts = [ txt.lower() for ( txt, label ) in data[train_size:-1] ]
test_labels = [ label for ( txt , label ) in data[train_size:-1] ]

emoji_data = pd.read_csv("C:\\Users\\Premnath\\Desktop\\MDSS\\MDDS 9\\emojis.csv")
emo_dict = {}
for index, row in emoji_data.iterrows():
    emo_dict[row["column2"]] = row["column1"]

#converting emojis to its respective emotions
def emo_convert(data):
    sp_data = data.split(" ")
    for i in sp_data:
        if i in emo_dict:
            sp_data[sp_data.index(i)] = emo_dict[i]
    data = " ".join(sp_data)
    return emoji.demojize(data)

train_texts = []
test_texts = []

#Preprocessing the data 
for i in emo_train_texts:
    train_texts.append(re.sub(r'[/\-()_,.:@#?!&$]', ' ', emo_convert(i)))
for i in emo_test_texts:
    test_texts.append(re.sub(r'[/\-()_,.:@#?!&$]', ' ', emo_convert(i)))
    
print(train_texts)
for i in range(0,len(train_labels)):
    if train_labels[i] == "happy":
        train_labels[i]=0
    elif train_labels[i] == "sad":
        train_labels[i]=1
    elif train_labels[i] == "angry":
        train_labels[i]=2
    elif train_labels[i] == "others":
        train_labels[i]=3

for i in range(0,len(test_labels)):
    if test_labels[i] == "happy":
        test_labels[i]=0
    elif test_labels[i] == "sad":
        test_labels[i]=1
    elif test_labels[i] == "angry":
        test_labels[i]=2
    elif test_labels[i] == "others":
        test_labels[i]=3


Using TensorFlow backend.
c:\python\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


['let me study do you have to pass your test to work in your field  yes', "right lol  only if you knew who i am talking about i don't know", 'had dinner  bfast for dinner makes whatever bfast you are eating so much better do u like mustang ', 'can u send u r pic to me i sent you one in a message now', 'thanks to u too bro thumbs up light skin tone  thanks bro  Happy nice talking to u', "no you r not tell me one reason why shouldn't i go to that fight   because i m sad ", "calling is more fun other than text you're telling me  yup", "dude     you're a friggin software as are you homosapien", 'shut up i love you so much  grinning face with big eyes  grinning face with big eyes  never ever message me again', 'going to tell you what  cause i told you what ', 'nautanki grinning face  grinning face  grinning face  grinning face  grinning face  great  thumbs up   winking face with tongue  good night', 'i love money and i love spending it  good one', 'tell me i am free makes you question    te

In [2]:
embeddings =  gensim.models.Word2Vec(train_texts, min_count=1, size=300)

tokenizer = Tokenizer(num_words=max_features,lower=True)
tokenizer.fit_on_texts(train_texts)
vocab_size = len(tokenizer.word_index) + 1
train_sequences = sequence.pad_sequences( tokenizer.texts_to_sequences( train_texts ) , maxlen=max_sent_len )
test_sequences = sequence.pad_sequences( tokenizer.texts_to_sequences( test_texts ) , maxlen=max_sent_len )
print(train_sequences)
train_matrix = tokenizer.texts_to_matrix( train_texts )
test_matrix = tokenizer.texts_to_matrix( test_texts )
embedding_weights = np.zeros( ( max_features , embeddings_dim ) )
for word,index in tokenizer.word_index.items():
  if index < max_features:
    try: embedding_weights[index,:] = embeddings[word]
    except: embedding_weights[index,:] = np.random.rand( 1 , embeddings_dim )

le = preprocessing.LabelEncoder( )
le.fit( train_labels + test_labels )

class_weights = class_weight.compute_class_weight('balanced',
                                                  np.unique(train_labels),
                                                  train_labels)
class_weights_dict = dict(zip(le.transform(list(le.classes_)),
                              class_weights))

train_labels = le.transform( train_labels )
test_labels = le.transform( test_labels )
print("Classi considerate: " + repr( le.classes_ ))

train_labels = to_categorical(train_labels)

np.random.seed(0)

"""pre-trained Glove"""
embeddings_index = dict()

# Glove
# f = open("C:\\Users\\Premnath\\Desktop\\MDSS\\embeddings\\glove.6B\\glove.6B.300d.txt", encoding="utf8")

# Fasttext
# f = open("C:\\Users\\Premnath\\Desktop\\MDSS\\embeddings\\crawl-300d-2M-subword\\crawl-300d-2M-subword.vec", encoding="utf8")

#glove web crawler
f = open("C:\\Users\\Premnath\\Desktop\\MDSS\\embeddings\\glove.840B.300d\\glove.840B.300d.txt", encoding="utf8")

for line in f:
    values = line.split()
#     word = values[0]
#     coefs = np.asarray(values[1:], dtype='float32')
    #for glove.840B.300d
    word = ''.join(values[:-300])
    coefs = np.asarray(values[-300:], dtype='float32')
    
    embeddings_index[word] = coefs
f.close()


embedding_matrix = np.zeros((vocab_size, 300))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

[[   0    0    0 ...   25 1864   34]
 [   0    0    0 ...    1   27   29]
 [   0    0  180 ...    7   32 5747]
 ...
 [   0    0    0 ...   43   25  864]
 [   0    0   16 ...  104   17   82]
 [   0    0    0 ...  489 1427   55]]


c:\python\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


Classi considerate: array([0, 1, 2, 3])


In [3]:
model = Sequential()
model.add(Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=20,trainable=False))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(64, activation='tanh', dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
model.add(BatchNormalization())
model.add(LSTM(64,activation='tanh', dropout=0.2, recurrent_dropout=0.2))
model.add(BatchNormalization())
model.add(Dense(num_classes))
model.add(Activation('softmax'))

adam=keras.optimizers.Adam(lr=0.001)
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

model.summary()


print("---CLASS WEIGHTS---")
class_weights_dict[3] = 1.75
print(class_weights_dict)

model.fit(train_sequences, train_labels , epochs=9, batch_size=40, class_weight=class_weights_dict)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 300)           3649800   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 20, 300)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 20, 64)            93440     
_________________________________________________________________
batch_normalization_1 (Batch (None, 20, 64)            256       
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                33024     
_________________________________________________________________
batch_normalization_2 (Batch (None, 64)                256       
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 260       
__________

In [4]:
# to get the probabilities of classes in result file
prob_result = model.predict_proba( test_sequences )

# predicting on the test data
results = model.predict_classes( test_sequences )
test_labels = list(test_labels)
results = list(results)
prob_result = list(prob_result)
# print(test_labels,results)
print ("confusion_matrix : ")
print ("Happy Sad Angry Others")
print (confusion_matrix(test_labels, results))
print ("classification_report: ")
print (classification_report(test_labels, results))
count = 0
f = open("C:\\Users\\Premnath\\Desktop\\MDSS\\MDDS 9\\results.txt", "w+",encoding="utf8", newline="\n")
f.write("Sentence" + "\t\t\t" + "expected label" + "\t" + "label obtained" + "\t" + "[Happy Sad Angry Others]" + "\n")
for i in range(0,len(results)):
    if test_labels[i] == results[i]:
        count = count+1;
    if test_labels[i] == 0:
        test_labels[i]="happy"
    elif test_labels[i] == 1:
        test_labels[i]= "sad"
    elif test_labels[i] == 2:
        test_labels[i]= "angry"
    elif test_labels[i] == 3:
        test_labels[i]= "others" 
    if results[i] == 0:
        results[i]="happy"
    elif results[i] == 1:
        results[i]= "sad"
    elif results[i] == 2:
        results[i]= "angry"
    elif results[i] == 3:
        results[i]= "others" 
    
    f.write(str(test_texts[i]) + "\t" + str(test_labels[i]) + "\t" + str(results[i]) + "\t" + str(prob_result[i]) + "\n")

f.write("number of correct: " + str(count) + " out of " + str(len(test_labels)) + "\n")

f.close()

confusion_matrix : 
Happy Sad Angry Others
[[1088   13    5  157]
 [   7 1499   36  128]
 [  19   66 1397  210]
 [ 194  135   86 4007]]
classification_report: 
             precision    recall  f1-score   support

          0       0.83      0.86      0.85      1263
          1       0.88      0.90      0.89      1670
          2       0.92      0.83      0.87      1692
          3       0.89      0.91      0.90      4422

avg / total       0.88      0.88      0.88      9047



In [5]:
#test dataset file without labels
final_data = pd.read_csv("C:\\Users\\Premnath\\Desktop\\MDSS\\MDDS 9\\devwithoutlabels.txt",sep='\t')
final_data['single'] = final_data['turn1']+" "+final_data['turn2']+" "+final_data['turn3']
# drop_features(['id','turn1','turn2','turn3'],test_data)
final_text = []
for i in list(final_data['single']):
    final_text.append(re.sub(r'[/\-()_,.:@#?!&$]', ' ', emo_convert(i)))
final_data_sequences = sequence.pad_sequences( tokenizer.texts_to_sequences( final_text ) , maxlen=max_sent_len )

#predicting the data based on the trained model
test_results = model.predict_classes( final_data_sequences )
test_results = list(test_results)
for i in range(len(test_results)):
    if test_results[i] == 0:
        test_results[i]= "happy"
    elif test_results[i] == 1:
        test_results[i]= "sad"
    elif test_results[i] == 2:
        test_results[i]= "angry"
    elif test_results[i] == 3:
        test_results[i]= "others" 
final_result = pd.DataFrame({'turn1':final_data['turn1'],'turn2':final_data['turn2'],'turn3':final_data['turn3']})
final_result['label'] = test_results
final_result.index.names=['id']
#saving the predicted results to a test file
final_result.to_csv('C:\\Users\\Premnath\\Desktop\\MDSS\\MDDS 9\\test.txt',sep='\t')